In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amananandrai/ag-news-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ag-news-classification-dataset


In [28]:
import pandas as pd
train_df=pd.read_csv("/kaggle/input/ag-news-classification-dataset/train.csv")
test_df=pd.read_csv("/kaggle/input/ag-news-classification-dataset/test.csv")

In [29]:
train_df

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [30]:
label_map={1:"World",2:"Sports",3:"Business",4:"Sci/Tech"}
train_df['category']=train_df['Class Index'].map(label_map)
test_df['category']=test_df['Class Index'].map(label_map)

In [31]:
train_df

,Class Index,Title,Description,category
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business
...,...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...,World
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...,Sports
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...,Sports
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...,Sports


In [32]:
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

stop_words=set(stopwords.words('english'))

def clean_text(text):
    text=BeautifulSoup(text,"html.parser").get_text()
    text=re.sub(r'http\S+|www\S+|https\S+','',text,flags=re.MULTILINE)
    text=re.sub(r'@\w+|#\w+','',text)
    text=re.sub(r'\d+','',text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text=text.lower()
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

train_df['text']=(train_df['Title']+" "+train_df['Description']).apply(clean_text)
test_df['text']=(test_df['Title']+" "+test_df['Description']).apply(clean_text)

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [35]:
train_df

,Class Index,Title,Description,category,text
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli...",Business,wall st bears claw back black reuters reuters ...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...,Business,carlyle looks toward commercial aerospace reut...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...,Business,oil economy cloud stocks outlook reuters reute...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...,Business,iraq halts oil exports main southern pipeline ...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco...",Business,oil prices soar alltime record posing new mena...
...,...,...,...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...,World,pakistans musharraf says wont quit army chief ...
119996,2,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...,Sports,renteria signing topshelf deal red sox general...
119997,2,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...,Sports,saban going dolphins yet miami dolphins put co...
119998,2,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...,Sports,todays nfl games pittsburgh ny giants time pm ...


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=TfidfVectorizer(ngram_range=(1,3),max_features=10000)
x_train=vectorizer.fit_transform(train_df['text'])
x_test=vectorizer.transform(test_df['text'])

y_train=train_df['category']
y_test=test_df['category']

In [60]:
x_train[0].toarray()[0]
for idx, value in enumerate(x_train[0].toarray()[0]):
    if value != 0:
        print(f"{vectorizer.get_feature_names_out()[idx]}: {value:.4f}")

back: 0.1793
bears: 0.2838
black: 0.2555
green: 0.2497
reuters: 0.2596
reuters reuters: 0.1748
seeing: 0.3136
st: 0.2206
streets: 0.2810
wall: 0.4483
wall st: 0.3433
wall streets: 0.3420


In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

LRmodel=LogisticRegression(max_iter=1000)
LRmodel.fit(x_train,y_train)
y_pred_LR=LRmodel.predict(x_test)
print(f"Classification Report",classification_report(y_test,y_pred_LR))

Classification Report               precision    recall  f1-score   support

    Business       0.88      0.89      0.89      1900
    Sci/Tech       0.89      0.88      0.89      1900
      Sports       0.95      0.98      0.96      1900
       World       0.93      0.91      0.92      1900

    accuracy                           0.91      7600
   macro avg       0.91      0.91      0.91      7600
weighted avg       0.91      0.91      0.91      7600



In [53]:
from sklearn.svm import LinearSVC

SVMmodel = LinearSVC()
SVMmodel.fit(x_train, y_train)
y_pred_svm = SVMmodel.predict(x_test)
print("Linear SVM Classification Report:\n", classification_report(y_test, y_pred_svm))


Linear SVM Classification Report:
               precision    recall  f1-score   support

    Business       0.88      0.88      0.88      1900
    Sci/Tech       0.89      0.89      0.89      1900
      Sports       0.95      0.98      0.96      1900
       World       0.93      0.90      0.92      1900

    accuracy                           0.91      7600
   macro avg       0.91      0.91      0.91      7600
weighted avg       0.91      0.91      0.91      7600



In [47]:
from sklearn.ensemble import RandomForestClassifier

RFmodel = RandomForestClassifier(n_estimators=100, random_state=42)
RFmodel.fit(x_train, y_train)
y_pred_rf = RFmodel.predict(x_test)
print("Random Forest Classification Report:\n", classification_report(y_test, y_pred_rf))


KeyboardInterrupt: 

In [63]:
def predict_category(text):
    cleaned = clean_text(text)
    vect = vectorizer.transform([cleaned])
    pred = SVMmodel.predict(vect)
    return pred[0]

print(predict_category("G7 Summit Ends with Focus on Climate and Global Security World leaders at the G7 summit emphasized the urgency of climate action and collective security in light of ongoing global tensions and natural disasters."))


World
